# <b><font color='lightgreen'> Python to VerilogA </font></b>

Having the model trained in the python framework is possible to acesse the weights and bias and other parameters of the network. This script provides functions capable to, given the input parameters, generate a verilogA code.

Work by: André Amaral & Nuno Lourenço (ICG-IT.Lx)

In [1]:
# Needed Libraries
import csv
import numpy as np

### <b><font color='blue'> Fully Connected Layers </font></b>


<b> How to get the weights and bias </b>

l = model.fc1.state_dict()

W = l['weight'].detach().cpu().numpy()

B = l['bias'].detach().cpu().numpy()


#### <b><font color=#27BFC1> FullConneted Layer </font></b>

In [3]:
def FC_HiddenLayer_to_VerilogA(W, B, input_dim, output_dim):

    # ---------------------------- Get the Weights Vector in VerilogA Structure ----------------------------

    Pesos_VA = []

    for i in range(W.shape[1]):
        Wa = W[:, i]

        middle = []
        first = '{%f' % Wa[0]

        if (W.shape[0] != 1):
            middle_out = ',%f' % Wa[1]
            middle = first + middle_out

        last = '}'

        if (W.shape[0] > 2):
            for i in (n+2 for n in range(np.size(W, 0)-2)):
                aux = ',%f' % Wa[i]
                middle = middle + aux
            Pesos_VA.append(middle + last)
        elif (W.shape[0] == 2):
            Pesos_VA.append(middle + last)
        elif (W.shape[0] == 1):
            Pesos_VA.append(first + last)

    # ------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------
    # ---------------------------- Get the Bias Vector in VerilogA Structure -------------------------------
    middle = []
    first = '{%f' %B[0]

    if(B.shape[0] != 1):
        middle_out = ',%f' %B[1]
        middle = first + middle_out

    last = '}'

    if (B.shape[0] > 2):
        for i in (n+2 for n in range(np.size(B,0)-2)):
            aux = ',%f' %B[i]
            middle = middle + aux
        b1_VA = middle + last
    elif (B.shape[0] == 2):
        b1_VA = middle + last
    elif (B.shape[0] == 1):
        b1_VA = first + last

    # ------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------
    # ------------------------------------ Write the Script of VerilogA ------------------------------------
    neurons_number_next = np.size(W,0)-1
    neurons_number_prev = np.size(W,1)-1

    with open('Middle_Layer.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module ANN_Middle_Layer(x,y);'])
        writer.writerow([''])
        writer.writerow(['  parameter integer x_dim = %i;' %input_dim])
        writer.writerow(['  parameter integer y_dim = %i;' %output_dim])
        
        for i in range(neurons_number_prev+1):
            writer.writerow(['  parameter real W2%i[0:%i] = %s;' %(i,neurons_number_next,Pesos_VA[i])])
        
        writer.writerow(['  parameter real b[0:%i] = %s;' %(neurons_number_next,b1_VA)])                
        writer.writerow(['  real a2[0:%i];' %neurons_number_next])
        writer.writerow([''])
        writer.writerow(['  input [0:%i]x;' %neurons_number_prev])
        writer.writerow(['  voltage [0:%i]x;' %neurons_number_prev])
        writer.writerow(['  output [0:%i]y;' %neurons_number_next])
        writer.writerow(['  voltage [0:%i]y;' %neurons_number_next])
        writer.writerow(['  genvar i;'])
        writer.writerow([''])
        writer.writerow(['  analog begin'])

        #Initialize the output variable (since is use an acumulator)
        for i in range(neurons_number_next+1):
            writer.writerow(['      a2[%i] = 0;' %i])
        
        writer.writerow([''])

        #Write the forward path
        for i in range(neurons_number_next+1):
            for j in range(neurons_number_prev+1):
                writer.writerow(['      a2[%i] = V(x[%i]) * W2%i[%i] + a2[%i];' %(i,j,j,i,i)])
        
        writer.writerow([''])

        #Update the output voltage pin
        writer.writerow(['      for(i = 0; i<= %i; i = i + 1) begin' %neurons_number_next])
        writer.writerow(['          V(y[i]) <+ a2[i] + b[i];'])

        writer.writerow(['      end'])
        writer.writerow(['  end'])
        writer.writerow(['endmodule'])

### <b><font color='blue'> LSTM Layers </font></b>


To get the internal weights and bias use:

<b>Weights Input:</b> <i>w_ii, w_if, w_ic, w_io = <model_name>.<lstm_layer_name>.weight_ih_l0.chunk(4, 0)</i>

<b>Weights Hidden:</b> <i>w_ii, w_if, w_ic, w_io = <model_name>.<lstm_layer_name>.weight_hh_l0.chunk(4, 0)</i>

In case of use DataParallel (train the model with multiple GPU's):

<b>Weights Input:</b> <i>w_ii, w_if, w_ic, w_io = <model_name>.<module><lstm_layer_name>.weight_ih_l0.chunk(4, 0)</i>

In [1]:
def LSTM_Layer_to_VerilogA():
    with open('LSTM_Layer.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module LSTM_Layer(H_prev, C_prev, x, H_out, C_out);'])
        writer.writerow(['  	input H_prev, C_prev, x;'])
        writer.writerow(['  	voltage H_prev, C_prev, x;'])
        writer.writerow(['  	output H_out, C_out;'])
        writer.writerow(['  	voltage H_out, C_out;'])
        writer.writerow([''])
        writer.writerow(['  	real C_til;'])
        writer.writerow(['  	real O_t_aux;'])
        writer.writerow(['  	real O_t;'])
        writer.writerow(['  	real I_t_aux;'])
        writer.writerow(['  	real I_t;'])
        writer.writerow(['  	real F_t_aux;'])
        writer.writerow(['  	real F_t;'])
        writer.writerow(['  	real C_t;'])
        writer.writerow(['  	real output_value;'])
        writer.writerow([''])
        writer.writerow(['  	parameter real Wf = ????;'])
        writer.writerow(['  	parameter real Wi = ????;'])
        writer.writerow(['  	parameter real Wo = ????;'])
        writer.writerow(['  	parameter real Wc = ????;'])
        writer.writerow(['  	parameter real Uf = ????;'])
        writer.writerow(['  	parameter real Ui = ????;'])
        writer.writerow(['  	parameter real Uo = ????;'])
        writer.writerow(['  	parameter real Uc = ????;'])
        writer.writerow(['  	parameter real bf = ????;'])
        writer.writerow(['  	parameter real bi = ????;'])
        writer.writerow(['  	parameter real bo = ????;'])
        writer.writerow(['  	parameter real bc = ????;'])
        writer.writerow([''])
        writer.writerow(['  	parameter real HiddenInit = ????;'])
        writer.writerow(['  	parameter real CellInit = ????;'])
        writer.writerow([''])
        writer.writerow(['  	analog begin'])
        writer.writerow([''])
        writer.writerow(['  	    V(H_prev) <+ Hidden;'])
        writer.writerow(['  	    V(H_prev) <+ Cell;'])
        writer.writerow([''])
        writer.writerow(['  	    C_til = tanh(Wc * V(x) + Uc * V(H_prev) + bc);'])
        writer.writerow([''])
        writer.writerow(['  	    O_t_aux = Wo * V(x) + Uo * V(H_prev) + bo;'])
        writer.writerow(['  	    O_t = 1/(1+exp(-O_t_aux));'])
        writer.writerow([''])
        writer.writerow(['  	    I_t_aux = Wi * V(x) + Ui * V(H_prev) + bi;'])
        writer.writerow(['  	    I_t = 1/(1+exp(-I_t_aux));'])
        writer.writerow([''])
        writer.writerow(['  	    F_t_aux = Wf * V(x) + Uf * V(H_prev) + bf;'])
        writer.writerow(['  	    F_t = 1/(1+exp(-F_t_aux));'])
        writer.writerow([''])
        writer.writerow(['  	    C_t = F_t * V(C_prev) + I_t * C_til;'])
        writer.writerow([''])
        writer.writerow(['  	    output_value = O_t * tanh(V(C_out));'])
        writer.writerow([''])
        writer.writerow(['  	end'])
        writer.writerow([''])
        writer.writerow(['  	V(C_out) <+ C_t;'])
        writer.writerow(['  	V(C_prev) <+ C_t;'])
        writer.writerow(['  	V(H_out) <+ output_value;'])
        writer.writerow(['  	V(H_prev) <+ output_value;'])
        writer.writerow([''])
        writer.writerow(['endmodule'])


### <b><font color='blue'> Activation Functions </font></b>


#### <b><font color=#27BFC1> ReLU </font></b>

In [ ]:
 # ------------------------------------ Write the Script of VerilogA ------------------------------------
def ReLU_to_VerilogA():
    with open('ReLU.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module ReLU(n_deact, n_act);'])
        writer.writerow(['  input n_deact;'])
        writer.writerow(['  output n_act;'])
        writer.writerow(['  eletrical n_deact, n_act;'])
        writer.writerow(['  real result;'])
        writer.writerow(['  real p_num;'])
        writer.writerow([' '])
        writer.writerow(['  analog begin'])
        writer.writerow(['      p_num = V(n_deact);'])
        writer.writerow(['      if(p_num >= 0)'])
        writer.writerow(['          result = p_num;'])
        writer.writerow(['      else'])
        writer.writerow(['          result = 0.0;'])
        writer.writerow(['      V(n_act) <+ result;'])
        writer.writerow(['  end'])
        writer.writerow(['endmodule;'])

#### <b><font color=#27BFC1> ELU </font></b>

In [ ]:
 # ------------------------------------ Write the Script of VerilogA ------------------------------------
def ELU_to_VerilogA():
    with open('ELU.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module ELU(n_deact, n_act);'])
        writer.writerow(['  input n_deact;'])
        writer.writerow(['  output n_act;'])
        writer.writerow(['  eletrical n_deact, n_act;'])
        writer.writerow(['  real result;'])
        writer.writerow(['  real p_num;'])
        writer.writerow(['  parameter real alpha = 1;'])
        writer.writerow([' '])
        writer.writerow(['  analog begin'])
        writer.writerow(['      p_num = V(n_deact);'])
        writer.writerow(['      if(p_num > 0)'])
        writer.writerow(['          result = p_num;'])
        writer.writerow(['      else'])
        writer.writerow(['          result = alpha * (exp(p_num) - 1);'])
        writer.writerow(['      V(n_act) <+ result;'])
        writer.writerow(['  end'])
        writer.writerow(['endmodule;'])